In [1]:
# Install required packages
!pip3 install folium
!pip3 install wget
!pip3 install pandas

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9687 sha256=f0e9529ed69b14e2d2a3875714472aa318ccd71cdd307c124b3003cb6ee2b48f
  Stored in directory: /Users/amriimamdeen/Library/Caches/pip/wheels/8a/b8/04/0c88fb22489b0c049bee4e977c5689c7fe597d6c4b0e7d0b6a
Successfully built wget


In [2]:
# Import libraries
import folium
import wget
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Download dataset
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df = pd.read_csv(spacex_csv_file)

# Select relevant columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

100% [............................................................] 7710 / 7710

In [3]:
# Create base map
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add markers for each launch site
for index, row in launch_sites_df.iterrows():
    site_coordinate = [row['Lat'], row['Long']]
    
    # Circle marker
    folium.Circle(
        location=site_coordinate,
        radius=50,
        color='#000000',
        fill=True,
        fill_color='#d35400'
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    
    # Text marker
    folium.map.Marker(
        site_coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

site_map

In [4]:
# Create marker cluster
marker_cluster = MarkerCluster()

# Function to assign marker colors
def assign_marker_color(launch_outcome):
    return 'green' if launch_outcome == 1 else 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Create map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
site_map.add_child(marker_cluster)

# Add markers
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map

In [5]:
# Distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Coordinates dictionary (example for CCAFS SLC-40)
proximities = {
    'Coastline': {'lat': 28.56367, 'lon': -80.568},
    'Highway': {'lat': 28.563, 'lon': -80.57085},
    'Railway': {'lat': 28.57206, 'lon': -80.58527},
    'City': {'lat': 28.10485, 'lon': -80.6458}  # Titusville
}

# Create map for CCAFS SLC-40
launch_site = launch_sites_df.iloc[0]
site_coordinate = [launch_site['Lat'], launch_site['Long']]
site_map = folium.Map(location=site_coordinate, zoom_start=13)

# Add launch site marker
folium.Marker(
    site_coordinate,
    popup=launch_site['Launch Site'],
    icon=folium.Icon(color='blue')
).add_to(site_map)

# Add proximity markers and lines
for name, coord in proximities.items():
    distance = calculate_distance(
        launch_site['Lat'], launch_site['Long'],
        coord['lat'], coord['lon']
    )
    
    # Distance marker
    distance_marker = folium.Marker(
        [coord['lat'], coord['lon']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{distance:.2f} KM</b></div>'
        )
    )
    
    # Connection line
    folium.PolyLine(
        locations=[site_coordinate, [coord['lat'], coord['lon']]],
        weight=2
    ).add_to(site_map)
    
    distance_marker.add_to(site_map)

site_map